In [33]:
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# 📋 Disease details dictionary
disease_info = {
    'Apple___Apple_scab': {'name': 'Apple Scab', 'precautions': 'Remove fallen leaves, apply fungicides (e.g., sulfur), prune for airflow.'},
    'Apple___Black_rot': {'name': 'Black Rot', 'precautions': 'Remove infected fruit/wood, apply fungicides, ensure proper sanitation.'},
    'Apple___Cedar_apple_rust': {'name': 'Cedar Apple Rust', 'precautions': 'Remove nearby cedar trees, apply fungicides, improve tree vigor.'},
    'Apple___healthy': {'name': 'Healthy Apple', 'precautions': 'Maintain regular pruning, monitor for pests, ensure balanced nutrition.'},
    'Blueberry___healthy': {'name': 'Healthy Blueberry', 'precautions': 'Keep soil acidic, prune regularly, monitor for pests.'},
    'Cherry_(including_sour)___Powdery_mildew': {'name': 'Powdery Mildew (Cherry)', 'precautions': 'Apply fungicides (e.g., sulfur), improve air circulation, avoid overhead watering.'},
    'Cherry_(including_sour)___healthy': {'name': 'Healthy Cherry', 'precautions': 'Prune for structure, monitor for diseases, maintain soil health.'},
    'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': {'name': 'Gray Leaf Spot (Corn)', 'precautions': 'Use resistant hybrids, rotate crops, apply fungicides if severe.'},
    'Corn_(maize)___Common_rust_': {'name': 'Common Rust (Corn)', 'precautions': 'Plant resistant varieties, apply fungicides early, ensure good field drainage.'},
    'Corn_(maize)___Northern_Leaf_Blight': {'name': 'Northern Leaf Blight (Corn)', 'precautions': 'Rotate crops, use resistant hybrids, apply fungicides during wet seasons.'},
    'Corn_(maize)___healthy': {'name': 'Healthy Corn', 'precautions': 'Rotate crops, monitor for pests, maintain soil fertility.'},
    'Grape___Black_rot': {'name': 'Black Rot (Grape)', 'precautions': 'Remove infected berries, apply fungicides, prune for ventilation.'},
    'Grape___Esca_(Black_Measles)': {'name': 'Black Measles (Grape)', 'precautions': 'Prune infected wood, improve irrigation, no effective chemical control.'},
    'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': {'name': 'Leaf Blight (Grape)', 'precautions': 'Apply fungicides, remove debris, ensure canopy airflow.'},
    'Grape___healthy': {'name': 'Healthy Grape', 'precautions': 'Prune regularly, monitor for pests, maintain balanced fertilization.'},
    'Orange___Haunglongbing_(Citrus_greening)': {'name': 'Citrus Greening', 'precautions': 'Remove infected trees, control psyllid vectors, use healthy planting material.'},
    'Peach___Bacterial_spot': {'name': 'Bacterial Spot (Peach)', 'precautions': 'Use resistant varieties, apply copper-based sprays, avoid overhead watering.'},
    'Peach___healthy': {'name': 'Healthy Peach', 'precautions': 'Prune for structure, monitor for pests, ensure proper irrigation.'},
    'Pepper,_bell___Bacterial_spot': {'name': 'Bacterial Spot (Pepper)', 'precautions': 'Use disease-free seeds, apply copper sprays, rotate crops.'},
    'Pepper,_bell___healthy': {'name': 'Healthy Pepper', 'precautions': 'Monitor for pests, maintain soil moisture, stake plants for support.'},
    'Potato___Early_blight': {'name': 'Early Blight (Potato)', 'precautions': 'Rotate crops, apply fungicides (e.g., chlorothalonil), remove infected debris.'},
    'Potato___Late_blight': {'name': 'Late Blight (Potato)', 'precautions': 'Apply fungicides, remove infected tubers, ensure good field drainage.'},
    'Potato___healthy': {'name': 'Healthy Potato', 'precautions': 'Rotate crops, use certified seeds, monitor for blight.'},
    'Raspberry___healthy': {'name': 'Healthy Raspberry', 'precautions': 'Prune canes, monitor for spider mites, maintain soil health.'},
    'Soybean___healthy': {'name': 'Healthy Soybean', 'precautions': 'Rotate crops, monitor for aphids, ensure proper nodulation.'},
    'Squash___Powdery_mildew': {'name': 'Powdery Mildew (Squash)', 'precautions': 'Apply fungicides (e.g., sulfur), improve air circulation, avoid wet foliage.'},
    'Strawberry___Leaf_scorch': {'name': 'Leaf Scorch (Strawberry)', 'precautions': 'Remove infected leaves, apply fungicides, ensure good drainage.'},
    'Strawberry___healthy': {'name': 'Healthy Strawberry', 'precautions': 'Use mulch, monitor for slugs, rotate planting beds.'},
    'Tomato___Bacterial_spot': {'name': 'Bacterial Spot (Tomato)', 'precautions': 'Use disease-free seeds, apply copper sprays, remove infected plants.'},
    'Tomato___Early_blight': {'name': 'Early Blight (Tomato)', 'precautions': 'Rotate crops, apply fungicides, stake plants for airflow.'},
    'Tomato___Late_blight': {'name': 'Late Blight (Tomato)', 'precautions': 'Apply fungicides (e.g., mancozeb), remove infected leaves, avoid wet conditions.'},
    'Tomato___Leaf_Mold': {'name': 'Leaf Mold (Tomato)', 'precautions': 'Improve ventilation, apply fungicides, avoid overhead watering.'},
    'Tomato___Septoria_leaf_spot': {'name': 'Septoria Leaf Spot (Tomato)', 'precautions': 'Remove infected leaves, apply fungicides, ensure crop rotation.'},
    'Tomato___Spider_mites Two-spotted_spider_mite': {'name': 'Spider Mites (Tomato)', 'precautions': 'Use miticides, spray water to dislodge mites, introduce predatory insects.'},
    'Tomato___Target_Spot': {'name': 'Target Spot (Tomato)', 'precautions': 'Apply fungicides, remove infected debris, improve air circulation.'},
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus': {'name': 'Yellow Leaf Curl Virus (Tomato)', 'precautions': 'Control whiteflies, use resistant varieties, remove infected plants.'},
    'Tomato___Tomato_mosaic_virus': {'name': 'Mosaic Virus (Tomato)', 'precautions': 'Use virus-free seeds, disinfect tools, remove infected plants.'},
    'Tomato___healthy': {'name': 'Healthy Tomato', 'precautions': 'Stake plants, monitor for pests, maintain even watering.'}
}

# 🧪 Class labels (should match training order)
class_names = sorted(disease_info.keys())

def predict_single_image(image_path, model_path, img_size=128):
    if not os.path.exists(image_path):
        print(f"❌ Image file not found: {image_path}")
        return

    if not os.path.exists(model_path):
        print(f"❌ Model file not found: {model_path}")
        return

    try:
        print(f"🔍 Loading model from {model_path}...")
        model = load_model(model_path)

        print(f"📸 Preprocessing image: {image_path}...")
        img = load_img(image_path, target_size=(img_size, img_size))
        img_array = img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        print("⏳ Running prediction...")
        start_time = time.time()
        predictions = model.predict(img_array)
        elapsed_time = time.time() - start_time

        predicted_index = np.argmax(predictions)
        confidence = predictions[0][predicted_index] * 100
        predicted_class = class_names[predicted_index]
        disease = disease_info[predicted_class]['name']
        precautions = disease_info[predicted_class]['precautions']

        print("\n✅ Prediction Completed!")
        print(f"📂 Image: {os.path.basename(image_path)}")
        print(f"🧪 Class Name: {predicted_class}")
        print(f"🌿 Disease Name: {disease}")
        # print(f"📊 Confidence: {confidence:.2f}%")
        print(f"⚠️ Precautions: {precautions}")
        # print(f"⏱️ Inference Time: {elapsed_time:.2f} seconds")

    except Exception as e:
        print("❌ An error occurred during prediction:")
        print(str(e))

# 🚀 Example usage
if __name__ == "__main__":
    image_path = "potato.webp"        # 🖼️ Your test image
    model_path = "./models/best_resnet50.h5"   # 🤖 Trained model
    predict_single_image(image_path, model_path)


🔍 Loading model from ./models/best_resnet50.h5...


📸 Preprocessing image: potato.webp...
⏳ Running prediction...
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step

✅ Prediction Completed!
📂 Image: potato.webp
🧪 Class Name: Corn_(maize)___Common_rust_
🌿 Disease Name: Common Rust (Corn)
⚠️ Precautions: Plant resistant varieties, apply fungicides early, ensure good field drainage.
